In [1]:
import datasets
from datasets import load_dataset

test_dataset = load_dataset("cnn_dailymail", '2.0.0', split='test[:5%]')


Reusing dataset cnn_dailymail (C:\Users\phill\.cache\huggingface\datasets\cnn_dailymail\2.0.0\2.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [39]:
test_dataset[3]['id']

'0095ce085581314285f894af73a55ea9ef003412'

In [29]:
i = 1
article, highlights = test_dataset[i]['article'], test_dataset[i]['highlights']

In [79]:
from transformers import AutoTokenizer
paraphrase_tokenizer = AutoTokenizer.from_pretrained(paraphrase_checkpoint)

https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to C:\Users\phill\.cache\huggingface\transformers\tmp09ko1a76
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 14.5kB/s]
storing https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/tokenizer_config.json in cache at C:\Users\phill/.cache\huggingface\transformers\7a2648c30292886df951b6c935c094a38b218647dd65f4790ce223bf2929b73e.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for C:\Users\phill/.cache\huggingface\transformers\7a2648c30292886df951b6c935c094a38b218647dd65f4790ce223bf2929b73e.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/config.json from cache at C:\Users\phill/.cache\huggingface\transformers\1d2fdd3792e3bcdf73c40c6d598a5c14b7626090fcbeb2f953c99390ea842c15.0

In [68]:
def tokenize_paraphrase_detection(a, b):
    return paraphrase_tokenizer(a, b, truncation=True, return_tensors='pt').to('cuda')
def tokenize_fn2(examples):
    return paraphrase_tokenizer(examples['article'], examples['highlights'], truncation=True, return_tensors='pt', padding=True)

In [77]:
from transformers.pipelines import pipeline
paraphrase_checkpoint = 'bert-base-cased-finetuned-mrpc'#'data/models/test_paraphrase_distilbert_1/checkpoint-3500'
#pipe = pipeline('text-classification', model= paraphrase_checkpoint)

In [70]:
wrong_answer = 'There is no way this could be the correct answer.'

In [78]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(paraphrase_checkpoint)

https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\phill\.cache\huggingface\transformers\tmpz0j647j_
Downloading: 100%|██████████| 433/433 [00:00<00:00, 433kB/s]
storing https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/config.json in cache at C:\Users\phill/.cache\huggingface\transformers\1d2fdd3792e3bcdf73c40c6d598a5c14b7626090fcbeb2f953c99390ea842c15.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
creating metadata file for C:\Users\phill/.cache\huggingface\transformers\1d2fdd3792e3bcdf73c40c6d598a5c14b7626090fcbeb2f953c99390ea842c15.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
loading configuration file https://huggingface.co/bert-base-cased-finetuned-mrpc/resolve/main/config.json from cache at C:\Users\phill/.cache\huggingface\transformers\1d2fdd3792e3bcdf73c40c6d598a5c14b7626090fcbeb2f953c99390ea842c15.0d87139f53a477d9f900f8a9

In [92]:
seq1 = 'The hunter shot what he thought was a deer, but it was actually a hay bail.'
seq2 = 'The hunter shot a hay bail on accident.'
seq3 = 'What are you even talking about?'

In [93]:
example = tokenize_paraphrase_detection(seq1, seq2)
wrong_example = tokenize_paraphrase_detection(seq1, seq3)

In [81]:
example = tokenize_paraphrase_detection(article, highlights)
wrong_example = tokenize_paraphrase_detection(article, wrong_answer)

In [ ]:
import torch

In [ ]:
model.to('cuda')

In [88]:
classes = ["not paraphrase", "is paraphrase"]

In [94]:
for i in range(10):
    article, highlights = test_dataset[i]['article'], test_dataset[i]['highlights']
    example = tokenize_paraphrase_detection(article, highlights)
    #wrong_example = tokenize_paraphrase_detection(article, wrong_answer)
    paraphrase_classification_logits = model(**example).logits
    paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
    for i in range(len(classes)):
        print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

not paraphrase: 81%
is paraphrase: 19%


In [95]:
paraphrase_classification_logits = model(**wrong_example).logits
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

not paraphrase: 96%
is paraphrase: 4%


In [41]:
from transformers import Trainer

trainer = Trainer(model=model)

In [53]:
list(mini_test[:5].keys())

['article', 'highlights', 'id', 'input_ids', 'attention_mask']

In [55]:
trainer.predict(mini_test)

***** Running Prediction *****
  Num examples = 574
  Batch size = 8


ValueError: too many values to unpack (expected 2)

In [12]:
result = pipe(tokenize_paraphrase_detection(article, highlights), num_workers=4, truncation=True)
print(result)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [6]:
result = pipe(article + '</s>' + wrong_answer, num_workers=4, truncation=True)
print(result)

[{'label': 'LABEL_0', 'score': 0.9978054165840149}]


In [6]:
print(result)

[{'label': 'LABEL_0', 'score': 0.9978054165840149}, {'label': 'LABEL_0', 'score': 0.9856269359588623}]
